In [1]:
from openeo.local import LocalConnection
local_conn = LocalConnection("./")

url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"
spatial_extent =  {"east": 11.40, "north": 46.52, "south": 46.46, "west": 11.25}
temporal_extent = ["2022-06-01", "2022-06-30"]
bands = ["red", "nir"]
properties = {"eo:cloud_cover": dict(lt=80)}
s2_datacube = local_conn.load_stac(
    url=url,
    spatial_extent=spatial_extent,
    temporal_extent=temporal_extent,
    bands=bands,
    properties=properties,
)

b04 = s2_datacube.band("red")
b08 = s2_datacube.band("nir")
ndvi = (b08 - b04) / (b08 + b04)
ndvi = ndvi.add_dimension(name="bands",label="NDVI",type="bands")
result_ndvi = ndvi.execute()

Did not load machine learning processes due to missing dependencies: Install them like this: `pip install openeo-processes-dask[implementations, ml]`
/home/mclaus@eurac.edu/anaconda3/envs/openeo/lib/python3.10/site-packages/openeo/metadata.py:264: UserWarning: No cube:dimensions metadata
  complain("No cube:dimensions metadata")


In [2]:
result_ndvi

<xarray.DataArray 'stackstac-0c6d32df684b642d54e6f118d8bc562e' (bands: 1,
                                                                time: 8,
                                                                y: 701, x: 1170)>
dask.array<broadcast_to, shape=(1, 8, 701, 1170), dtype=float64, chunksize=(1, 1, 580, 933), chunktype=numpy.ndarray>
Coordinates: (12/51)
  * time                                     (time) datetime64[ns] 2022-06-02...
    id                                       (time) <U24 'S2B_32TPS_20220602_...
  * x                                        (x) float64 6.726e+05 ... 6.843e+05
  * y                                        (y) float64 5.155e+06 ... 5.148e+06
    s2:granule_id                            (time) <U62 'S2B_OPER_MSI_L2A_TL...
    s2:product_type                          <U7 'S2MSI2A'
    ...                                       ...
    proj:transform                           object {0, 600000, 10, 5200020, ...
    gsd                                      int64 10
    proj:shape                               object {10980}
    raster:bands                             object {'nodata': 0, 'data_type'...
    epsg                                     int64 32632
  * bands                                    (bands) <U4 'NDVI'
Attributes:
    spec:                           RasterSpec(epsg=32632, bounds=(600000.0, ...
    crs:                            epsg:32632
    transform:                      | 10.00, 0.00, 600000.00|\n| 0.00,-10.00,...
    resolution:                     10.0
    reduced_dimensions_min_values:  {'band': 0}

In [3]:
from raster2stac import Raster2STAC

In [4]:
rs2stac = Raster2STAC(result_ndvi,t_dim="time",b_dim="bands",output_folder="result_NDVI")

In [5]:
rs2stac.generate_stac()


ts: 2022-06-02T10:17:58.886000000
b: NDVI


/home/mclaus@eurac.edu/anaconda3/envs/openeo/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))



ts: 2022-06-10T10:28:04.746000000
b: NDVI

ts: 2022-06-12T10:18:02.133000000
b: NDVI

ts: 2022-06-15T10:27:59.095000000
b: NDVI

ts: 2022-06-17T10:18:10.538000000
b: NDVI

ts: 2022-06-20T10:28:07.343000000
b: NDVI

ts: 2022-06-25T10:28:00.461000000
b: NDVI

ts: 2022-06-27T10:18:12.018000000
b: NDVI


In [7]:
cube = local_conn.load_stac("result_NDVI/20220627101812/metadata.json")

In [8]:
cube.execute()

<xarray.DataArray 'stackstac-48712e6ff2f2f92aab56d6b8e303ef56' (time: 1,
                                                                band: 1,
                                                                y: 702, x: 1171)>
dask.array<fetch_raster_window, shape=(1, 1, 702, 1171), dtype=float64, chunksize=(1, 1, 702, 1024), chunktype=numpy.ndarray>
Coordinates: (12/16)
  * time                 (time) datetime64[ns] 2022-06-27T10:18:12.018000
    id                   (time) <U14 '20220627101812'
  * band                 (band) <U4 'NDVI'
  * x                    (x) float64 6.726e+05 6.726e+05 ... 6.843e+05 6.843e+05
  * y                    (y) float64 5.155e+06 5.155e+06 ... 5.148e+06 5.148e+06
    proj:epsg            int64 32632
    ...                   ...
    proj:shape           object {1170, 701}
    raster:bands         object {'data_type': 'float64', 'scale': 1.0, 'offse...
    common_name          object None
    center_wavelength    object None
    full_width_half_max  object None
    epsg                 int64 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(672580.0, 5147610.0, 684290.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 672580.00|\n| 0.00,-10.00, 5154630.00|\n| 0.0...
    resolution:  10.0